In [3]:
#Import requirements:
import cv2 as cv


# The Pong Project
Christian Kuate, Aissata Sylla, Raphaela NKOUDOU, Paul Béglin

### Présentation synthétique du projet


### Répartition des tâches
Nous avons décidé de séparer le projet en 2 parties: 
1. le jeu (avec comme controles les "UP ARROW" "DOWN ARROW" normaux, (que Christian et Aissata auraient du faire)
2. les contrôles fait par une main, grâce à OpenCV : fait par Paul 
3. le Menu fait par Raphaela

Puis nous allons intégrer ces 3 parties ensemble

### Initialisation
#### Git & GitHub
Afin d'apprendre à utiliser git (un des outils les plus importants dans notre carrière de développeurs) nous allons tout faire sur GitHub en essayant d'adopter les bonnes pratiques, comme créer une branche pour chaque feature avant de merge avec main.

#### Environnement Virtuel Python
De plus, à cause de problèmes d'environement qui empêchait de faire fonctionner OpenCV, nous avons créé des environnements virtuels afin de séparer la version Python qu'utilise le système et la version dont nous avons besoin avec toutes nos dépendences. On a notamment utilisé Conda.

...


### Tests fonctionnels

OpenCV & cvZone pour les contrôles du jeu
J'ai décidé d'utiliser une bibliothèque additionelle qui est basée sur OpenCV, notamment car elle traite le HandTracking de manière très efficace ce qui permet au code d'être plus simple et lisible, mais aussi de s'éxecuter plus rapidement. 

In [ ]:
#Premier tests caméra avec opencv et cvzone Hand Tracking
#Attention, dans Jupyternotebooks on ne peut pas sortir de GUI, donc afin d'avoir quelque chose qui s'affiche inline
#j'ai changé le code pour utiliser matplotlibe et avoir chaque image qui s'affiche inline... 
#MAIS il faut arrêter le processus rapidement, sinon vous allez avoir pleins d'images 

import cv2 as cv
from cvzone.HandTrackingModule import HandDetector
from matplotlib import pyplot as plt

liveVideo = cv.VideoCapture(0)
handDetector = HandDetector(detectionCon = 0.8)

while True:
    ret, frame = liveVideo.read()

    #Show the image with matplotlib
    plt.imshow(frame)
    plt.show()
    #cv.imshow('frame', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):    #waitKey(1) & 0xFF is a bitwise operation to only keep the last 8 bits and compare it to ord('q')
        break

# After the loop release the cap object
liveVideo.release()
# Destroy all the windows
cv.destroyAllWindows()

On utilise la point de l'index comme réference pour le mouvement (le point numéro 8 de la main sur OpenCV)
c'est pourquoi on appelle: 

```yCoordinate = lmList1[8][1:2]```

Pour avoir ses coordonnées

In [ ]:
while True:
    ret, frame = liveVideo.read()
    frame = cv.flip(frame, 1)
    hands, frame = handDetector.findHands(frame, flipType=False)
#for each hand we'Ll have info Like Hand--›dict{lmList,boundinqboundary, img)
    if hands:
        hand1=hands[0]                          #gives us first hand
        lmList1=hand1["lmList"]                 #List of 21 Landmarks
        bbox1=hand1["bbox"]                     #x,y,w,h of bounding box
        centerPoint1=hand1[ "center" ]          #center of the hand cx, cy
        handType1=hand1["type"]                 #Left or right
        finger1=handDetector.fingersUp(hand1)
        
        #needs something to find the distance / x,y positions 
        #length, info, frame=handDetector.findDistance(lmList1[8], lmList1[12])
        yCoordinate = lmList1[8][1:2]       
        #yCoordinate = tuple(yCoordinate) #careful change data type to tuple with tuple() if doing a circle bounding box
        #cv2.circle() ...
        print(yCoordinate)

LA CRÉATION DE LA PAGE D'ACCEUIL
Pour réaliser la page d'accueil nous avons utilisé la bibliothéque pygame en utilisant le concept de multi-fenetres. A cet effet nous avons donc créer cinq fonctions principales qui constituent les quatres boutons de l'interface et la fonction main qui va se charger d'afficher ces boutons à l'écran.
le principe est simple, le clic d'un bouton correspond à l'appel de la fonction
Egalement, pour generer ces boutons à l'ecran, nous avons opté pour la création d'une classe BUTTON. Ce qui rend la géneration des boutons facile et non redondante.

La classe BUTTON:
    le def _init_
    Elle posséde un constructeur qui va initialiser les différents attributs des boutons(image, position du bouton, text, la police, la couleur, et la couleur secondaire) et trois méthodes

In [ ]:
class Button():
	def __init__(self, image, pos, text_input, font, base_color, hovering_color):
		self.image = image
		self.x_pos = pos[0]
		self.y_pos = pos[1]
		self.font = font
		self.base_color, self.hovering_color = base_color, hovering_color
		self.text_input = text_input
		self.text = self.font.render(self.text_input, True, self.base_color)
		if self.image is None:
			self.image = self.text
		self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))
		self.text_rect = self.text.get_rect(center=(self.x_pos, self.y_pos))

    la methode update
    elle permet d'afficher le bouton à l'ecran d'accueil une fois qu'elle est appelée

In [ ]:
def update(self, screen):
		if self.image is not None:
			screen.blit(self.image, self.rect)
		screen.blit(self.text, self.text_rect)

la methode checkForInput
elle permet de vérifier si u utilisateur a cliqué sur un bouton

In [ ]:
def checkForInput(self, position):
		if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
			return True
		return False

la methode changeColor
Elle permet de changer de couleur en réaction au clic de la souris

In [ ]:
def changeColor(self, position):
		if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
			self.text = self.font.render(self.text_input, True, self.hovering_color)
		else:
			self.text = self.font.render(self.text_input, True, self.base_color)

La fonction Play
Il s'agit du main.py
C'est grace à elle que le jeu se lance. Nous reviendrons dessus un peu plus bas

La fonction aide
C'est dans cette fonction que s'effectue l'affichage des régles du jeu.
Une fois que t'utilisateur clique sur le bouton help, une fenetre différente du menu d'accueil s'ouvre et présente les régles pour etre mesure de jouer

In [ ]:
def aide():
    while True:
        HELP_MOUSE_POS = pygame.mouse.get_pos()

        fenetre.fill("white")
        titleFont  =  pygame.font.Font( 'image jeu/font.ttf' , 100 )
       
        texteRegles  =  pygame.font.Font( 'image jeu/font.ttf'  , 25 )

        # zones cliquables
        

        #affichage à l'écran
        fenetre.blit ( titleFont . render ( "Règles" , True , ( 0 , 0 , 0 )), ( 370, 30 ))
        bouton_back = Button(image=None, pos=(640, 540), 
                            text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        fenetre.blit ( texteRegles . render ( 'Le ping pong est une sorte de tennis de table ' , True , ( 0 , 0 , 0 )), ( 20 , 150  ))
        fenetre.blit ( texteRegles . render ( "en mouvement constant. L'un gagne si l 'autre ne  ", True , ( 0 , 0 , 0 )), ( 20, 200 ))
        fenetre.blit ( texteRegles . render ( "parvient pas à intercepter la balle de l'autre" , True , ( 0 , 0 , 0 )), ( 20 , 250 ))
        fenetre.blit ( texteRegles . render ( 'ici les déplacements sont controlés par les mou' , True , ( 0 , 0 , 0 )), ( 20 , 300 ))
        fenetre.blit ( texteRegles . render ( 'vements du bras. si le joueur souhaite se déplacer' , True , ( 0 , 0 , 0 )), ( 20 , 350 ))
        fenetre.blit ( texteRegles . render ( "vers le haut, il souleve la main et s'il souhaite se"  , True , ( 0 , 0 , 0 )), ( 20 , 400 ))
        fenetre.blit ( texteRegles . render ( 'deplacer vers le bas, il baisse sa main' , True , ( 0 , 0 , 0 )), ( 20 , 450 ))
    

        

        bouton_back.changeColor(HELP_MOUSE_POS)
        bouton_back.update(fenetre)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if bouton_back.checkForInput(HELP_MOUSE_POS):
                  main_menu()

        pygame.display.update()    


## Menu & Jeu

Nous avons commencé en séparant les fonctions et classes dans différents fichiers, mais par manque d'expertise, nous avons finalement tout regroupé dans le même fichier main, avec comme seule séparation le fichier menu.

Les autres fichiers sont laissés là si vous voulez voir notre cheminement.

Il y a plein de code inutile, même à l'intérieur des fichiers menu et main. Notamment car nous avons été plusieurs à programmer et que nous n'avons pas assez bien combiné ensemble le travail.

En intégrant le jeu dans le menu après avoir cliqué sur play, la fenêtre ne se resize / scale pas correctement.


### Bilan collectif et individuel





### Bibliographie (nos sources)
- https://code.visualstudio.com/docs/sourcecontrol/overview
- https://training.github.com/downloads/github-git-cheat-sheet.pdf
- https://stackoverflow.com/questions/7974849/how-can-i-make-one-python-file-run-another
- https://docplayer.fr/174641349-Introduction-a-pygame.html
- https://www.youtube.com/watch?v=lA5MHT1PbIA 
- https://stackoverflow.com/questions/35372700/whats-0xff-for-in-cv2-waitkey1 

